# **Home Credit Default Risk**  
This project attempts to model and predict the default risk of consumer loan applicants by analyzing and extracting data from several sources including bureau data as well as previous home credit applications and installments. I divided the work into the following steps / sections:

## 1. Exploratory Data Analysis 
#### 1a. Baseline Models
\- Run logistic regression / random forest baseline models to gauge expectations and get an initial impression on main application file and the target variable  
#### 1b. Clean Data  
\- Investigate and impute null values as well as anomalies across main application file and supplemental files
#### 1c. Feature Exploration  
\- Explore features by analyzing correlations and relationship with target variable (default / no default)  
## 2. Feature Engineering
#### 2a. Supplemental Data  
\- Engineer features by grouping and flattening supplemental datasets  
#### ***2b. Application Data***
\- ***Merge supplemental data with main application dataframe. Create additional features through a combination of manual calculations with domain knowledge as well as polynomial features*** 
## 3. Gradient Boosting Classifier
#### 3a. Train LightGBM  
\- Train LightGBM model using data and previously engineered features  
#### 3b. Model Results / Analysis  
\- Compare models and assess feature importances
  

<hr>

## __2b. Feature Engineering - Application Data__

#### __Sections__
2b1. Load application data, then encode all features where applicable  
2b2. Calculate polynomial features then append to application dataset  
2b3. Load all aggregated supplemental data and merge with main application dataframe  

<hr>

In [1]:
import pandas as pd
import numpy as np
import os
import warnings
import pickle

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import Imputer, MinMaxScaler, LabelEncoder, StandardScaler, PolynomialFeatures
from functools import reduce

from rq_config import project_3_path

warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
%matplotlib  inline

In [2]:
pkl_dir = os.path.join(project_3_path,'data/','pkl/')

### __2b1. Load application data, then encode all features where applicable__

In [3]:
application = pd.read_pickle(pkl_dir + 'application_train.pkl')

In [4]:
# Manually calculate credit annuity ratio

application['CREDIT_ANNUITY_RATIO'] = application['AMT_CREDIT'] / application['AMT_ANNUITY']

In [5]:
# Manually calculate credit to income ratio

application['CREDIT_INCOME_RATIO'] = application['AMT_CREDIT'] / application['AMT_INCOME_TOTAL']

In [6]:
# Separate categorical features for processing

numeric_features_application = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']

categorical_features_application = application.select_dtypes(exclude = numeric_features_application)

In [7]:
# One-hot encoding for binary variables

le = LabelEncoder()
binary_features = []
remaining_categoricals = []
for feature in categorical_features_application:
    # Identify features that can be one-hot encoded (includes null values)
    n_uniques = len(list(application[feature].unique()))
    
    if n_uniques <= 2:
        le.fit(application[feature])
        application[feature] = le.transform(application[feature])
        binary_features.append(feature)
        print(f'One-hot encoded: {feature}')
    else:
        remaining_categoricals.append(feature)

One-hot encoded: NAME_CONTRACT_TYPE
One-hot encoded: FLAG_OWN_CAR
One-hot encoded: FLAG_OWN_REALTY
One-hot encoded: AMT_ANNUITY_ANOMALY
One-hot encoded: DAYS_EMPLOYED_ANOMALY
One-hot encoded: OWN_CAR_AGE_ANOMALY
One-hot encoded: CNT_FAM_MEMBERS_ANOMALY
One-hot encoded: OBS_30_CNT_SOCIAL_CIRCLE_ANOMALY
One-hot encoded: OBS_60_CNT_SOCIAL_CIRCLE_ANOMALY
One-hot encoded: AMT_REQ_CREDIT_BUREAU_QRT_ANOMALY


In [8]:
print(f'Remaining features with more than one class {remaining_categoricals}')

Remaining features with more than one class ['CODE_GENDER', 'NAME_TYPE_SUITE', 'NAME_INCOME_TYPE', 'NAME_EDUCATION_TYPE', 'NAME_FAMILY_STATUS', 'NAME_HOUSING_TYPE', 'OCCUPATION_TYPE', 'WEEKDAY_APPR_PROCESS_START', 'ORGANIZATION_TYPE', 'FONDKAPREMONT_MODE', 'HOUSETYPE_MODE', 'WALLSMATERIAL_MODE', 'EMERGENCYSTATE_MODE']


In [9]:
# Create separate dataframe of dummy encoded variables then merge with original dataframe while dropping original remaining categorical variables

application_dummies = pd.get_dummies(application.loc[:,remaining_categoricals])

application = application.join(application_dummies)

application = application.drop(columns = remaining_categoricals)

In [10]:
# Separate dummy features, then combine with binary categorical features

dummy_features = application_dummies.columns.tolist()

categorical_features_application = binary_features + dummy_features

In [11]:
# Load categorical features from supplemental datasets then append application categorical features. Save for gradient boosting model

categorical_features_supplemental = pickle.load(open(pkl_dir + 'categorical_features_supplemental.pkl','rb'))

categorical_features = categorical_features_application + categorical_features_supplemental

print(len(categorical_features))
    
pickle.dump(categorical_features,open(pkl_dir + 'categorical_features.pkl','wb'))

500


### __2b2. Calculate polynomial features then append to application dataset__

#### Select features to be combined based on feature importances and correlations from EDA and baseline models:

In [12]:
poly_features = application[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3', 'DAYS_BIRTH','DAYS_ID_PUBLISH','DAYS_REGISTRATION','TARGET']]

In [13]:
# Imputer for handling missing values

imputer = Imputer(strategy = 'median')

poly_target = poly_features['TARGET']
poly_features = poly_features.drop(columns = ['TARGET'])
poly_features = imputer.fit_transform(poly_features)
                                  
# Create polynomial object
poly_transformer = PolynomialFeatures(degree = 3)

In [14]:
poly_transformer.fit(poly_features)

# Transform the features
poly_features = poly_transformer.transform(poly_features)
print('Polynomial Features shape: ', poly_features.shape)

Polynomial Features shape:  (307511, 84)


In [15]:
# Extract combined feature names / labels

poly_labels = (poly_transformer.
               get_feature_names(input_features = ['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3', 'DAYS_BIRTH','DAYS_ID_PUBLISH','DAYS_REGISTRATION','TARGET']))

In [16]:
application_poly_features = pd.DataFrame(poly_features,columns = poly_labels)

#### Polynomial combined features from application data:

In [17]:
application_poly_features.head()

,1,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,DAYS_BIRTH,DAYS_ID_PUBLISH,DAYS_REGISTRATION,EXT_SOURCE_1^2,EXT_SOURCE_1 EXT_SOURCE_2,EXT_SOURCE_1 EXT_SOURCE_3,EXT_SOURCE_1 DAYS_BIRTH,EXT_SOURCE_1 DAYS_ID_PUBLISH,EXT_SOURCE_1 DAYS_REGISTRATION,EXT_SOURCE_2^2,EXT_SOURCE_2 EXT_SOURCE_3,EXT_SOURCE_2 DAYS_BIRTH,EXT_SOURCE_2 DAYS_ID_PUBLISH,EXT_SOURCE_2 DAYS_REGISTRATION,EXT_SOURCE_3^2,EXT_SOURCE_3 DAYS_BIRTH,EXT_SOURCE_3 DAYS_ID_PUBLISH,EXT_SOURCE_3 DAYS_REGISTRATION,DAYS_BIRTH^2,DAYS_BIRTH DAYS_ID_PUBLISH,DAYS_BIRTH DAYS_REGISTRATION,DAYS_ID_PUBLISH^2,DAYS_ID_PUBLISH DAYS_REGISTRATION,DAYS_REGISTRATION^2,EXT_SOURCE_1^3,EXT_SOURCE_1^2 EXT_SOURCE_2,EXT_SOURCE_1^2 EXT_SOURCE_3,EXT_SOURCE_1^2 DAYS_BIRTH,EXT_SOURCE_1^2 DAYS_ID_PUBLISH,EXT_SOURCE_1^2 DAYS_REGISTRATION,EXT_SOURCE_1 EXT_SOURCE_2^2,EXT_SOURCE_1 EXT_SOURCE_2 EXT_SOURCE_3,EXT_SOURCE_1 EXT_SOURCE_2 DAYS_BIRTH,EXT_SOURCE_1 EXT_SOURCE_2 DAYS_ID_PUBLISH,EXT_SOURCE_1 EXT_SOURCE_2 DAYS_REGISTRATION,EXT_SOURCE_1 EXT_SOURCE_3^2,EXT_SOURCE_1 EXT_SOURCE_3 DAYS_BIRTH,EXT_SOURCE_1 EXT_SOURCE_3 DAYS_ID_PUBLISH,EXT_SOURCE_1 EXT_SOURCE_3 DAYS_REGISTRATION,EXT_SOURCE_1 DAYS_BIRTH^2,EXT_SOURCE_1 DAYS_BIRTH DAYS_ID_PUBLISH,EXT_SOURCE_1 DAYS_BIRTH DAYS_REGISTRATION,EXT_SOURCE_1 DAYS_ID_PUBLISH^2,EXT_SOURCE_1 DAYS_ID_PUBLISH DAYS_REGISTRATION,EXT_SOURCE_1 DAYS_REGISTRATION^2,EXT_SOURCE_2^3,EXT_SOURCE_2^2 EXT_SOURCE_3,EXT_SOURCE_2^2 DAYS_BIRTH,EXT_SOURCE_2^2 DAYS_ID_PUBLISH,EXT_SOURCE_2^2 DAYS_REGISTRATION,EXT_SOURCE_2 EXT_SOURCE_3^2,EXT_SOURCE_2 EXT_SOURCE_3 DAYS_BIRTH,EXT_SOURCE_2 EXT_SOURCE_3 DAYS_ID_PUBLISH,EXT_SOURCE_2 EXT_SOURCE_3 DAYS_REGISTRATION,EXT_SOURCE_2 DAYS_BIRTH^2,EXT_SOURCE_2 DAYS_BIRTH DAYS_ID_PUBLISH,EXT_SOURCE_2 DAYS_BIRTH DAYS_REGISTRATION,EXT_SOURCE_2 DAYS_ID_PUBLISH^2,EXT_SOURCE_2 DAYS_ID_PUBLISH DAYS_REGISTRATION,EXT_SOURCE_2 DAYS_REGISTRATION^2,EXT_SOURCE_3^3,EXT_SOURCE_3^2 DAYS_BIRTH,EXT_SOURCE_3^2 DAYS_ID_PUBLISH,EXT_SOURCE_3^2 DAYS_REGISTRATION,EXT_SOURCE_3 DAYS_BIRTH^2,EXT_SOURCE_3 DAYS_BIRTH DAYS_ID_PUBLISH,EXT_SOURCE_3 DAYS_BIRTH DAYS_REGISTRATION,EXT_SOURCE_3 DAYS_ID_PUBLISH^2,EXT_SOURCE_3 DAYS_ID_PUBLISH DAYS_REGISTRATION,EXT_SOURCE_3 DAYS_REGISTRATION^2,DAYS_BIRTH^3,DAYS_BIRTH^2 DAYS_ID_PUBLISH,DAYS_BIRTH^2 DAYS_REGISTRATION,DAYS_BIRTH DAYS_ID_PUBLISH^2,DAYS_BIRTH DAYS_ID_PUBLISH DAYS_REGISTRATION,DAYS_BIRTH DAYS_REGISTRATION^2,DAYS_ID_PUBLISH^3,DAYS_ID_PUBLISH^2 DAYS_REGISTRATION,DAYS_ID_PUBLISH DAYS_REGISTRATION^2,DAYS_REGISTRATION^3
0,1.0,0.083037,0.262949,0.139376,-9461.0,-2120.0,-3648.0,0.006895,0.021834,0.011573,-785.612748,-176.038371,-302.918857,0.069142,0.036649,-2487.756636,-557.451017,-959.236466,0.019426,-1318.634256,-295.476654,-508.442846,89510521.0,20057320.0,34513728.0,4494400.0,7733760.0,13307904.0,0.000573,0.001813,0.000961,-65.234900,-14.617692,-25.153463,0.005741,0.003043,-206.575767,-46.289042,-79.652087,0.001613,-109.495390,-24.535485,-42.219552,7.432682e+06,1.665499e+06,2.865915e+06,3.732013e+05,6.421880e+05,1.105048e+06,0.018181,0.009637,-654.152107,-146.580960,-252.229879,0.005108,-346.733022,-77.695170,-133.694331,2.353667e+07,5.274044e+06,9.075336e+06,1.181796e+06,2.033581e+06,3.499295e+06,0.002707,-183.785678,-41.182289,-70.864618,1.247560e+07,2.795505e+06,4.810378e+06,6.264105e+05,1.077899e+06,1.854800e+06,-8.468590e+11,-1.897623e+11,-3.265344e+11,-4.252152e+10,-7.316910e+10,-1.259061e+11,-9.528128e+09,-1.639557e+10,-2.821276e+10,-4.854723e+10
1,1.0,0.311267,0.622246,0.535276,-16765.0,-291.0,-1186.0,0.096887,0.193685,0.166614,-5218.396475,-90.578788,-369.163031,0.387190,0.333073,-10431.950422,-181.073521,-737.983489,0.286521,-8973.906339,-155.765389,-634.837633,281065225.0,4878615.0,19883290.0,84681.0,345126.0,1406596.0,0.030158,0.060288,0.051861,-1624.316241,-28.194216,-114.908384,0.120520,0.103675,-3247.125160,-56.362268,-229.710137,0.089185,-2793.283699,-48.484674,-197.604203,8.748642e+07,1.518553e+06,6.189018e+06,2.635843e+04,1.074264e+05,4.378274e+05,0.240927,0.207254,-6491.237078,-112.672233,-459.207109,0.178286,-5583.975307,-96.924

#### Merge polynomial features with original application data:

In [18]:
# Include only polynomial features

application_merged = pd.concat([application,application_poly_features.iloc[:,7:-1]],axis = 1)

### __2b3. Load all aggregated supplemental data and merge with main application dataframe__

In [19]:
bureau = pd.read_pickle(pkl_dir + 'df_bureau_agg.pkl')
cc_balance = pd.read_pickle(pkl_dir + 'df_cc_balance_agg.pkl')
installments = pd.read_pickle(pkl_dir + 'df_installments_agg.pkl')
pos = pd.read_pickle(pkl_dir + 'df_pos_agg.pkl')
previous = pd.read_pickle(pkl_dir + 'df_previous_agg.pkl')

In [20]:
# Append to a list all dataframes that will be merged

merge_dataframes = [application_merged,bureau,cc_balance,installments,pos,previous]

In [21]:
# Merge all dataframes on SK_ID_CURR identifiers - will take a while

merged = reduce(lambda left,right: 
                pd.merge(left,right,on = ['SK_ID_CURR'],how = 'outer')
                ,merge_dataframes)

In [22]:
# Drop all extra SK_ID_CURR that don't have target

merged = merged[merged['TARGET'].notnull()].reset_index(drop = True)

In [23]:
merged.shape

(307511, 996)

In [24]:
merged.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,TOTALAREA_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,AMT_ANNUITY_ANOMALY,DAYS_EMPLOYED_ANOMALY,OWN_CAR_AGE_ANOMALY,CNT_FAM_MEMBERS_ANOMALY,OBS_30_CNT_SOCIAL_CIRCLE_ANOMALY,OBS_60_CNT_SOCIAL_CIRCLE_ANOMALY,AMT_REQ_CREDIT_BUREAU_QRT_ANOMALY,CREDIT_ANNUITY_RATIO,CREDIT_INCOME_RATIO,CODE_GENDER_F,CODE_GENDER_M,CODE_GENDER_XNA,NAME_TYPE_SUITE_Children,NAME_TYPE_SUITE_Family,NAME_TYPE_SUITE_Group of people,NAME_TYPE_SUITE_Other_A,NAME_TYPE_SUITE_Other_B,"NAME_TYPE_SUITE_Spouse, partner",NAME_TYPE_SUITE_Unaccompanied,NAME_INCOME_TYPE_Businessman,NAME_INCOME_TYPE_Commercial associate,NAME_INCOME_TYPE_Maternity leave,NAME_INCOME_TYPE_Pensioner,NAME_INCOME_TYPE_State servant,NAME_INCOME_TYPE_Student,NAME_INCOME_TYPE_Unemployed,NAME_INCOME_TYPE_Working,NAME_EDUCATION_TYPE_Academic degree,NAME_EDUCATION_TYPE_Higher education,NAME_EDUCATION_TYPE_Incomplete higher,NAME_EDUCATION_TYPE_Lower secondary,NAME_EDUCATION_TYPE_Secondary / secondary special,NAME_FAMILY_STATUS_Civil marriage,NAME_FAMILY_STATUS_Married,NAME_FAMILY_STATUS_Separated,NAME_FAMILY_STATUS_Single / not married,NAME_FAMILY_STATUS_Unknown,NAME_FAMILY_STATUS_Widow,NAME_HOUSING_TYPE_Co-op apartment,NAME_HOUSING_TYPE_House / apartment,NAME_HOUSING_TYPE_Municipal apartment,NAME_HOUSING_TYPE_Office apartment,NAME_HOUSING_TYPE_Rented apartment,NAME_HOUSING_TYPE_With parents,OCCUPATION_TYPE_Accountants,OCCUPATION_TYPE_Cleaning staff,OCCUPATION_TYPE_Cooking staff,OCCUPATION_TYPE_Core staff,OCCUPATION_TYPE_Drivers,OCCUPATION_TYPE_HR staff,OCCUPATION_TYPE_High skill tech staff,OCCUPATION_TYPE_IT staff,OCCUPATION_TYPE_Laborers,OCCUPATION_TYPE_Low-skill Laborers,OCCUPATION_TYPE_Managers,OCCUPATION_TYPE_Medicine staff,OCCUPATION_TYPE_Private service staff,OCCUPATION_TYPE_Realty agents,OCCUPATION_TYPE_Sales staff,OCCUPATION_TYPE_Secretaries,OCCUPATION_TYPE_Security staff,OCCUPATION_TYPE_Waiters/barmen staff,WEEKDAY_APPR_PROCESS_START_FRIDAY,WEEKDAY_APPR_PROCESS_START_MONDAY,WEEKDAY_APPR_PROCESS_START_SATURDAY,WEEKDAY_APPR_PROCESS_START_SUNDAY,WEEKDAY_APPR_PROCESS_START_THURSDAY,WEEKDAY_APPR_PROCESS_START_TUESDAY,WEEKDAY_APPR_PROCESS_START_WEDNESDAY,ORGANIZATION_TYPE_Advertising,ORGANIZATION_TYPE_Agriculture,ORGANIZATION_TYPE_Bank,ORGANIZATION_TYPE_Business Entity Type 1,ORGANIZATION_TYPE_Business Entity Type 2,ORGANIZATION_TYPE_Business Entity Type 3,ORGANIZATION_TYPE_Cleaning,ORGANIZATION_TYPE_Construction,ORGANIZATION_TYPE_Culture,ORGANIZATION_TYPE_Electricity,ORGANIZATION_TYPE_Emergency,ORGANIZATION_TYPE_Government,ORGANIZATION_TYPE_Hotel,ORGANIZATION_TYPE_Housing,ORGANIZATION_TYPE_Industry: type 1,ORGANIZ

In [25]:
# Save

merged.to_pickle(pkl_dir + 'merged.pkl')